In [1]:
import os
os.chdir('../../')
os.getcwd()

'/home/maxspad/proj/nlp-qual-max'

In [2]:
import pandas as pd
import mlflow
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
import src.models.train as train
from omegaconf import DictConfig, OmegaConf
from pprint import pprint
import pickle

### Identify Model to Use

In [3]:
EXPERIMENT_NAME = 'hparam_simplification'
RUN_ID = '8a2417c0ca784c12a4cccd8abca64a58'

### Define Paramteres

In [4]:
TEST_FILE_NAME = 'test.pkl'
TEST_FILE_DIR = 'data/processed/'
MLFLOW_DIR = 'mlruns/8'

### Load Data

In [5]:
df = pd.read_pickle(TEST_FILE_DIR + TEST_FILE_NAME)

In [6]:
df.columns

Index(['Unnamed: 0', 'Survey N', 'Question N', 'commentId', 'dataSource',
       'NumberFromRawData', 'comment', 'rating', 'q1p1', 'q1p2', 'q2p1',
       'q2p2', 'q3p1', 'q3p2', 'q1p1T', 'q1p2T', 'q2p1T', 'q2p2T', 'q3p1T',
       'q3p2T', 'P1QualScore', 'P2QualScore', 'Q1Match', 'Q2Match', 'Q3Match',
       'perfectMatch', 'RobMacCommentModified', 'RobMacQ1', 'RobMacQ2',
       'RobMacQ3', 'RobMacQualScore', 'Q1', 'Q2', 'Q3', 'QUAL', 'GenderRes',
       'GenderFac', 'Type', 'ObserverType', 'EPA', 'PGY', 'comment_spacy'],
      dtype='object')

### Load Model

In [7]:
with open(f'{MLFLOW_DIR}/{RUN_ID}/artifacts/model/model.pkl', 'rb') as f:
    clf = pickle.load(f)

## Calculate Test Metrics

In [11]:
y = df['Q2'].values
# invert
y = (y - 1) * -1
p = clf.predict(df[['comment_spacy']].values)
s = clf.decision_function(df[['comment_spacy']].values)

### Test Metrics

In [12]:
print(train.calculate_metrics(y, p, s))

{'balanced_accuracy': 0.7752202330207445, 'accuracy': 0.8346666666666667, 'roc_auc': 0.8667235009946008, 'f1': 0.8956228956228957, 'precision': 0.9236111111111112, 'recall': 0.869281045751634, 'tp': 47, 'tn': 266, 'fp': 22, 'fn': 40}
